In [1]:
import pandas as pd
import os
filepath = os.path.abspath('') + '/data-from-samples.csv'

In [2]:
spreadsheet = pd.read_csv(filepath)

In [3]:
spreadsheet.head(3)

,Sample,Far,High,Low,length,width,depth
0,copper,0,0.010,-0.008,32.48mm,9.47mm,9.48mm
1,aluminum,0,-0.009,0.015,31.92mm,9.47mm,9.4mm
2,titanium,0,-0.131,0.133,32.86mm,9.55mm,9.5mm
